![book header](header.png)

# Module 2 - Reading KITT sensor data

KITT can only drive autonomously if it is able to sense its environment. Two types of sensors are provided: 
(i) Two distance sensors mounted on the front of KITT, and 
(ii) Five microphones located around the field which can record the audio transmitted by the beacon on top of KITT and send it to the PC.

The idea of this task is to understand how to read out distance sensors in front of KITT to avoid obstacles and reading information from the microphones placed in the field. 

**Learning Objectives** The following is learned and practiced in this module:
- How to read out the distance sensors in front of KITT to detect obstacles
- How to read out the data from the microphones located around the field

**Preparation**
- Read this module.
- ASIO4ALL installation.
- Windows 10 users make sure that you install ASIO4ALL and make sure you use a build of PyAudio that supports ASIO ( @Mano should we mention the versions exactlly). Of course needless to say, your Python IDE needs to be ready!
- Make sure Python IDE is ready.

**What is needed**
- KITT
- Laptop/PC running Python
- Access to a field with a microphone setup and an audio card.

## The Distance Sensors

KITT has two ultrasonic sensors on the front left and right. These ultrasonic distance sensors consists of the SRF02 module, the datasheet of this module is also on Brightspace (@B update with links). The module works by transmitting a pulse train at 40 kHz and then listening to its echo. The time until the first echo is received is measured and converted to a distance in centimeters. According to the SRF02 datasheet, the time between two observations (the cycle period) has to be at least 66 ms. The modules are connected to the MCU, and the interface is pre-programmed. The cycle time is fixed at 70 ms, and in this period, the left and right sensors are started one after the other. The measurements are stored in a buffer on the MCU. A new measurement overwrites the older one. 

To have a good understanding of the limitations of the system, you have to determine the working of the ultrasonic sensors. The practical realization of the SRF02 modules is simple and adequate for its purpose, which is for parking sensors. The accuracy of the estimated distance is affected by many factors, such as the mounting of the sensors in combination with the type of beams they generate and the environment. 

Moreover, the distance measured by the sensors must be relayed to the computer’s control system. The various communication delays are at the origin of additional errors like KITT having already moved some distance before the new distance measurement reaches your PC.

**Status Commands:** The distance measurements are included in the output of the status request command, 

`serial_port.write(b'S\n')`

, but an isolated version containing only the distances can be obtained
by using:

`serial_port.write(b'Sd\n')`

Make sure to receive the data from KITT with the following: 

`serial_port.read_until(b'\x04')`

Similarly, `b'Sv\n'` makes KITT send the voltage of the battery pack, and `b'V\n'` shows version information of KITT. Again the data sent by KITT should be received with, 

`serial_port.read_until(b'\x04')`

## Distance reading Assignment 

In Module 4, you develop a car model which includes dynamics (velocity, response to driving commands) (??@B adapt to final updates). To develop this, you will need plots of position versus time. When driving on a straight line towards a wall, you can derive position from the parking sensor data. Further, in the final challenge, you will have to detect obstacles that could be in the path of KITT using the distance sensors. In this assignment, you should add sensor reading methods to the KITT class from module 1 also copied below. The received data should not just be stored in bulk but in a convenient format for later recall.  (@ Mano @Mehrdad should we be more clear here what we mean?)

(??@Mano how should we change this assignment now that we will give them the car model)

**Tasks**

Using the previously explained commands, have KITT transmit its various pieces of information. You can do this by adding a `read_sensors` method to the KITT class. Assign a key to show the distances measured. Display all the received data in an organized way in the terminal. (?? @Mano isn't it a task by itself? shouold we use the simulator for that?)

In [ ]:
## (@Mano @Mehrdad shall we add the kitt class here again with the new methods and ask them to complete it )




**Task 1: Static measurements** 

Perform measurements with the vehicle at stand-still. Introduce various obstacle configurations. Determine the accuracy of the sensors, their maximal range, and the field of view (beam angle).

**Task 2: Delay estimates**

Make an estimate of the delays for getting an update on the distance, and figure out how they impact the performance of the control chain. The question is: how old is an estimate by the time you receive it? Some delays you can determine from the data sheets, and some delays
you can measure using timing functions as described in Section 2.6.2 (??@B adapt to final changes).

**Task 3: Dynamic Measurements**

Perform distance measurements with KITT in motion (driving to the wall) and analyze the shape of the distance versus time. Is the plot continuous? Look at the plot for left and right sensor values, what do you see? Can you verify the 70 ms cycle time? Can you use these plots to estimate velocity accurately?

**Task 4: Measurement Data analysis and interpretation**

Analyze the measured data with an eye on the possibility of compensating for the possible errors. Implement some strategies to calibrate KITT.

```{note}{Bonus Tasks - Optional extension} Click to Show

It is advisable to store all the old distance data in a list inside the KITT class. This will be convenient during the final challenge, where the route planning might need old measurements to determine the position of objects. It will also help with characterizing the distance sensors. A good way of doing this is by initializing an empty list in the __init__ phase. Then, every time the sensor is read, you append this list with [time, left_dist, right_dist].

```

## Mid-term assessment 2.1 and report

After you finish this assignment, and ultimo in week 4, showcase the functionality of your script to your assigned TA. After you pass this assessment, you are ready to document your results in your midterm report. For this Module, you would include a chapter that covers the above tasks (using independently-readable text, i.e., don’t refer to “Task 1”). Include plots; for each plot it should be clear how the plot was made (i.e., the corresponding experimental set-up), and you have to describe what is seen in the plot before you discuss results and derive conclusions. Review the guidelines for more information. Include the corresponding code in an Appendix. Remember to document your code, using comments to define input/output variables of functions and to explain the logic and any modifications made. Your completed script will be crucial for the upcoming challenges, contributing to the overall autonomous driving system.



## The Microphones

Four microphones of the field are installed around its corners, and a fifth microphone is placed between two of the microphones at the edge of the field, at a level higher than the other four. These microphones, along with the beacon atop KITT, will be used to locate KITT within the field (chapter 5 ??@B update).

The appropriate sound card driver must be used to use the microphone array. The soundcard used in EPO-4 is a PreSonus AudioBox 1818VSL. On Linux, OSX (except the ARM version), and Windows 11, the sound card works out of the box. On Windows 10, it is necessary to install [ASIO4ALL](https://www.asio4all.org/) and a build of [PyAudio](https://www.lfd.uci.edu/~gohlke/pythonlibs/#pyaudio) compiled with ASIO support. If you use the Pipfile provided on Brightspace, the correct version of PyAudio should already be installed, but you still need to install ASIO4ALL manually.

A typical laptop will have many possible audio devices, for example the built-in microphone, a Bluetooth headset, and the AudioBox 1818VSL used in EPO-4. To initialize the microphone array, the correct audio device should be specified. This short script will list the index and names of all audio devices visible to PyAudio.

In [ ]:
# (??@B change this to a better location)
%pip install -r requirements.txt
%pip install keyboard
%pip install pyaudio

In [ ]:
import matplotlib
import numpy
import serial
import keyboard
import scipy
import statsmodels
import ipywidgets
import pyaudio
import tkinter as tk

In [ ]:

import pyaudio
pyaudio_handle = pyaudio.PyAudio()
for i in range(pyaudio_handle.get_device_count()):
    device_info = pyaudio_handle.get_device_info_by_index(i)
    print(i, device_info['name'])



```{important} Click to Show

On Windows 10, make sure to use the audio device index that has ASIO in the name. Other audio devices may be using the legacy MME or WDM Windows drivers, which may not support more than 2 synchronous
audio channels.

```

The microphone array must first be initialized. When doing so, the sampling frequency that will be used must be specified. This sampling frequency will either be 48 kHz or 44.1 kHz, depending on the type of audio device. Initializing the microphone array at device index `device_index` with a sampling frequency of `Fs` is done as, 


In [ ]:
# From the list of devices, add the desired index and the appropriate Fs value for the same.

stream = pyaudio_handle.open(input_device_index=device_index,
                            channels=5,
                            format=pyaudio.paInt16,
                            rate=Fs,
                            input=True)


To make a recording, the length of the recording must be specified. This must be specified as the number of audio frames to be recorded. The result will be a *bytes* object. Each audio frame will contain 5 samples, one for each microphone. Each sample contains 2 bytes, since we specified 16-bit audio. So, the return value of recording N frames is 10N bytes. To get a recording of N frames, one can run the
following command:


In [ ]:
samples = stream.read(N)

# To convert it into a numpy array
import numpy as np
data = np.frombuffer(samples, dtype='int16')

At this point, the microphone data is interleaved: data[0] contains the first sample of microphone 0, data[1] contains the first sample of microphone 1, data[2] contains the first sample of microphone 2, and so on until data[5] contains the second sample of microphone 0. Table 4.1 explains the concept more visually. This interleaved data stream should be deinterleaved into 5 streams, one for each microphone.

| data[0]      | data[1]      | data[2]      | data[3]      | data[4]      | data[5]      | data[6]      | data[7]      | ... |
|--------------|--------------|--------------|--------------|--------------|--------------|--------------|--------------|-----|
| mic 0        | mic 1        | mic 2        | mic 3        | mic 4        | mic 0        | mic 1        | mic 2        | ... |
| frame 0      | frame 0      | frame 0      | frame 0      | frame 0      | frame 1      | frame 1      | frame 1      | ... |


```{admonition} 

You may be familiar with the Matplotlib Python module, which can be used to plot the audio data received from the microphones. Matplotlib produces great-looking publication-ready figures. But drawing these plots can be slow. If you want to plot audio data in real time, consider using a more low-level GUI library such as PyGame or Pyglet and trade off beauty for speed. That said, Matplotlib supports some real-time features, such as animations. Just be aware that Matplotlib is not the only solution.

```

**Lab rules regarding the microphone array:** Many groups will be using the same setup, and to avoid
making other groups crazy, you are not allowed to rearrange the microphone connectors. Please also
don’t touch the volume settings. If these need to be adjusted, contact a TA.

## Microphone Recording Assignment 

The final part of communicating with KITT is using the 5 microphones. This will form an important part of the final challenge, a good implementation is thus essential. Again, you should add a method like `record` into the KITT class, with an input N. This should turn on the beacon, make a recording of N sec, deconvolve the recording into its separate channels, store it for later processing, and turn off the beacon. Make a function to visualize the recordings; this will prove valuable in debugging. (?? @Mano already include this in the the kitt class, encomment and complete? also add the simulator to check? both functions)

### Assignment

**Task 1** 

Initialize the microphone array and record one of your team members clapping near the microphones one after another. Separate the data stream of each microphone from the interleaved data. Plot the data of all five channels, allowing you to identify which channel of your recording represents which microphone.

**Task 2**

Turn on KITT’s beacon and record your results. Can you see the waveform of the transmission? Compare the waveform of the recording to an ideal OOK of your code. What can you see and what do you infer from this?

**Task 3**

Repeat the setup of Task 2, putting KITT nearer to one microphone than to others. Can you derive from the waveforms near which microphone KITT was placed? Show the plots you made and discuss your results and conclusions derived from them.

*Bonus Tasks - Optional*

- See if you can automate selecting the correct PyAudio device index. The correct device index changes from one computer to another and can sometimes even change on the same computer after a reboot. So, it is worth your time to make a program that can automatically select the right device index.
- Implement start-up sanity checks: some process which you can run after you arrive at the test field, so that you can quickly check the microphone connections and audio levels.
- Explore PyAudio’s callback mode. This manual describes what is called ‘blocking mode’. The stream.read() function will block your program until the requested number of frames has been received from the sound card. You can instead specify a callback function to process new audio frames as they arrive. If done carefully, this will allow your program to respond faster to new microphone samples, and enable you to drive while recording. You can read more about call-back mode in the official [PyAudio documentation](https://people.csail.mit.edu/hubert/pyaudio/docs/)


### Mid-term assessment 2.2 and report

After you finish this assignment, and ultimo in week 4, showcase the functionality of your script to your
assigned TA. After you pass this assessment, you are ready to document your results in your midterm
report.

For this Module, you would include a chapter that covers the above tasks (using independently-readable
text, i.e., don’t refer to “Task 1”). Include plots; for each plot it should be clear how the plot was made
(i.e., the corresponding experimental set-up), and you have to describe what is seen in the plot before
you discuss results and derive any conclusions. Be sure to answer the questions posed along with the
plots (using independently-readable text).

Include the corresponding code in an Appendix. Remember to document your code, using comments
to define input/output variables of functions and to explain the logic and any modifications made. Your
completed script will be crucial for the upcoming challenges, contributing to the overall autonomous
driving system.

This concludes the mid-term assignments related to communication with KITT. After the mid-term, you
must integrate this module with the localization module created by your colleagues. Take into account
that integrating is often harder than originally anticipated, e.g. your code has to run in parallel, and you
have to worry about timing aspects. Hopefully, using the KITT class will provide you with a sturdy and
flexible framework to continue your work towards the final challenge



(??@Mano is it worth to add the simulator here or let them just work with the real car? can even use the simulator?)

Extras
**Time Duration**
Two lab sessions and two preparation/reporting sessions at home.